# Building and Submitting Argo Workflows Using Hera

In [2]:
from hera.workflows import Steps, Step, Workflow, script, WorkflowsService
from hera.workflows.models import WorkflowTemplateRef, TemplateRef
from hera.shared import global_config

In [1]:
# token is obtained from https://workflows.argo.cuahsi.io/userinfo
argo_token = "v2:eyJhbGciOiJSU0EtT0FFUC0yNTYiLCJlbmMiOiJBMjU2R0NNIiwiemlwIjoiREVGIn0.Ltyir_rKyc6LtTOddzI90ybwg4uC11N7rdVJ_Sk0C5rguHAEvjkbOZC4pttOdlIjPHKrc8Ucbtf0IUGKzTZVlDZWXvCUqKyi7QL5qY7OZQwyOkdem1AYfu_dtlPEeMY9t-gei1Z5GGPloR0Qh-D8WvbADUda3onWlwkySm15YorCMNYlmhQSnRwhy_h5Z1cYRLSxFCcVUZ5V7ceU7cnSzOVC_LeQcWJBCNhCkHL1jQtZMG7GOULsHQZf5ZEQsFfuqFut0C1dn4rjnWilOyQSK0ScXWIX9oQqpPzg5p6BWx3HJzvtu1yEbfbdh6T-HXRUDIKyG7ZCKNAR88uD0Qq-eA.T94_tqVMAMAHsrc3.trsCc0BFqlCD0ACSUz-WJiGHoYAxtYbRmS93j1BZIhDveCVQe6rPXxtSZVO1d_MeQ-CoCTmVXk4StE0frsz6uezC-58cbv9q.C-pSavoLfhell9L1cdWmZA"
argo_host  = "https://workflows.argo.cuahsi.io"
argo_ns    = "workflows"



In [3]:
ws=WorkflowsService(host=argo_host,
                   token=argo_token,
                   namespace=argo_ns)

Test the connection

In [4]:
ws.get_info()

InfoResponse(links=None, managed_namespace=None, modals=None, nav_color=None)

Define a function to print job status

In [5]:
import time
import datetime 

def display_status(workflow):
    job_name = workflow.name

    # initial values
    finished_at = None
    start_time = None
    
    print(f'Job: {job_name}') 
    while finished_at is None:
        # query the job status
        status = workflow.workflows_service.get_workflow(job_name).status

        # set start time
        if start_time is None:
            start_time = status.started_at.__root__

        # get current time
        now = datetime.datetime.now(datetime.timezone.utc)

        # display progress
        print(f'Progress: {status.progress} ' +
              f'[{status.phase}]' + 
              f' - Elapsed time={now-start_time} seconds', end='\r')

        # update finished time
        finished_at = status.finished_at
    
        time.sleep(1)
        
    print('')
    print(f'Finished at: {str(finished_at.__root__)}')

## Run a sample job

Create a simple job (https://github.com/argoproj-labs/hera#examples) 

In [ ]:
@script()
def echo(message: str):
    print(message)
    
with Workflow(
    generate_name="hello-world-",
    entrypoint="steps",
    namespace="workflows",
    workflows_service=ws
) as w:
    with Steps(name="steps"):
        
        echo(arguments={"message": "Hello world!"})


Display the YAML that was generated

In [ ]:
print(w.to_yaml())

Submit the job

In [ ]:
_ = w.create()

Display the job status

In [ ]:
display_status(w)

## Run a Job for a Pre-defined Template

List workflow templates that exist in our cluster.

In [6]:
for wt in ws.list_workflow_templates().items:
    print (wt.metadata.name)

collect-aorc-forcing-batch
collect-aorc-forcing-full-v1.1
collect-aorc-forcing-v1
collect-aorc-forcing-v1.1
huc-to-shp
linked-ngen-metadata
map-reduce-aorc
metadata-extractor
metadata-extractor-with-dist
ngen-subset-v1
ngen-v2.0-subset-minio
nwm1-subset-minio
nwm2-subset-minio
parflow-subset-v1-by-huc-minio
serenity-now
test-dask
test-entrypoint
test-input-artifacts
test-map-reduce


Create a workflow template reference from one of the templates listed above

In [7]:
wt_ref = WorkflowTemplateRef(name="metadata-extractor", cluster_scope=True)

Create a new workflow using this template

In [13]:
with Workflow(
    generate_name="template-test-",
    entrypoint="tasks",
    workflows_service=ws,
    namespace='workflows',
) as w:
    metadata_extract_ref = TemplateRef(
        name="metadata-extractor",
        template="metadata-extractor",
    )
    with Steps(name="tasks") as s:
        Step(
            name='call-metadata-extract',
            template_ref=metadata_extract_ref,
            arguments={'bucket': 'subsetter-outputs',
                        'path': 'acastronova/testing-with-sepehr',
                       'output-path':'acastronova/testing-with-sepehr/metadata'
                       },
            )

Display the YAML that was created.

In [14]:
print(w.to_yaml())

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: template-test-
  namespace: workflows
spec:
  entrypoint: tasks
  templates:
  - name: tasks
    steps:
    - - arguments:
          parameters:
          - name: bucket
            value: subsetter-outputs
          - name: path
            value: acastronova/testing-with-sepehr
          - name: output-path
            value: acastronova/testing-with-sepehr/metadata
        name: call-metadata-extract
        templateRef:
          name: metadata-extractor
          template: metadata-extractor



Submit the job

In [15]:
_ = w.create()

Display the job status

In [16]:
display_status(w)

Job: template-test-6h52q
Progress: 1/1 [Succeeded] - Elapsed time=0:00:10.071083 seconds
Finished at: 2023-12-13 19:59:54+00:00
